In [4]:
import numpy as np
import sys
sys.path.append('../../')
sys.path.append('../../pyfos/')
from pyfos.models.feedforward import build_feedforward
from pyfos.generic.optimizer import gd_optimizer
from pyfos.generic.regularizers import gaus_prior
from functools import partial 
from pyfos.tc.simple import  simple_tc
from pyfos.tc.cv import  cv_tc
from pyfos.hyperoptimizers.random_search import random_optimize
from pyfos.hyperoptimizers.greed_optimize import greed_optimize
import theano
%matplotlib inline
import matplotlib.pylab as plt
import random
import cPickle

In [5]:
test_size=20
X_train = np.load('../../../data/linear2_x.npy').astype(theano.config.floatX)[:test_size]
Y_train = np.load('../../../data/linear2_y.npy').astype(theano.config.floatX)[:test_size]

X_test = np.load('../../../data/linear2_x.npy').astype(theano.config.floatX)[test_size:]
Y_test = np.load('../../../data/linear2_y.npy').astype(theano.config.floatX)[test_size:]
bias,var = np.load('../../../data/linear2_bias_var.npy')

X = np.arange(-2, 2, 0.1)
XX = np.vstack([ X,np.ones(X.shape[0]), X**2, X**3, X**4, X**5, X**6, X**7, X**8, X**9,  np.cos(X), np.sin(X)])
XX = XX.T
XX = (XX-bias)/var
XX[:,1] =  np.ones(X.shape[0])
XX = np.vstack([ XX.tolist(), X_train.tolist() , X_test.tolist()])

XX=np.array(sorted(XX.tolist(), key=lambda x:x[0])) 


In [6]:
inits = [0.0]
param_num = X_train.shape[1]
alphas = theano.shared(np.random.randn(12).astype(theano.config.floatX))
lr = theano.shared(np.array(7.5*10**(-3)).astype(theano.config.floatX))

optimizer = partial(gd_optimizer, learning_rate=lr)

model_build = partial(build_feedforward,  use_softmax=False,  structure = [X_train.shape[1], 1],   init_sigmas=inits, nonlinearity=lambda x:x, log_alphas =alphas, bias=False)
tc = cv_tc(model_build, optimizer,X_train, Y_train, batch_size=test_size, k=1)
tc.train_indices=[range(20)]
tc.validation_indices=[range(20,40)]
optimizer = partial(gd_optimizer, learning_rate=lr)


In [7]:
params = []

for i in xrange(1000):
    tc.do_train()
    if i%50 == 0:
        params.append(tc.models[0].params.eval())
    

In [5]:
params = np.array(params)


In [27]:
alphas.set_value(np.array([1]*12))

In [28]:
greed_optimize(partial(cv_tc, k=1, batch_size=20)
               , model_build, optimizer, 2, 20, 1000, np.vstack([X_train, X_test]), 
               np.hstack([Y_train, Y_test]), [alphas],lr=10**(-1), verbose=100
              )
print alphas.eval()

iteration 0, internal loss=-369.788721413, time=0.000293016433716
iteration 100, internal loss=-361.205834779, time=0.00985383987427
iteration 200, internal loss=-361.160671867, time=0.00940299034119
iteration 300, internal loss=-361.135979053, time=0.00941300392151
iteration 400, internal loss=-361.11933476, time=0.00945401191711
iteration 500, internal loss=-361.107625707, time=0.0281059741974
iteration 600, internal loss=-361.099116767, time=0.0135388374329
iteration 700, internal loss=-361.092777322, time=0.0114130973816
iteration 800, internal loss=-361.087967296, time=0.0105559825897
iteration 900, internal loss=-361.084270409, time=0.00936484336853
trial 0 [array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])]
iteration 0, internal loss=-369.788721413 hyperparam loss=-23.6687552753 time = 0.000367879867554
iteration 100, internal loss=-361.265883057 hyperparam loss=-32.2274397371 time = 0.0230348110199
iteration 200, internal loss=-361.121836562 hyperparam loss=-3